<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/Guilherme_Pereira/Aula_8_Guilherme_Pereira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = 'Guilherme Pereira'
print(f'Meu nome é {nome}')

Meu nome é Guilherme Pereira


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 60.5 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 50.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook

import torch.nn.functional as F

In [ ]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 13:33:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.examples = []
        for text in tqdm_notebook(texts):
            token_ids = tokenize(text=text, tokenizer=tokenizer)
            if len(token_ids) < context_size + 1:
                continue
            # Compute n-grams:
            for i in range(len(token_ids) - context_size):
                input_ids = token_ids[i:i + context_size]
                target_id = token_ids[i + context_size]
                self.examples.append((input_ids, target_id)) 

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return torch.LongTensor(input_ids), target_id

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 13:33:43--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.187.128, 64.233.188.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.187.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   147MB/s    in 0.8s    

2022-05-25 13:33:44 (147 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [ ]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


In [ ]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.

        super().__init__()
        
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.WQ = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.WK = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.WV = nn.Linear(embedding_dim, embedding_dim, bias=False) 
        self.WO = nn.Linear(embedding_dim, embedding_dim, bias=False)

        self.fc1    = nn.Linear(context_size*embedding_dim, embedding_dim*4)
        self.fc2    = nn.Linear(embedding_dim*4, embedding_dim*2)
        self.output = nn.Linear(embedding_dim*2,vocab_size)


    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.

        x = self.embeddings(inputs)

        q = self.WQ(x).to(device) #Shape = (B,L,D)
        k = self.WK(x).to(device) #Shape = (B,L,D)
        v = self.WV(x).to(device) #Shape = (B,L,D)

        X  = torch.matmul(F.softmax(torch.matmul(q, k.transpose(2,1)), dim=-1), v)

        X = self.WO(X)

        X = X + x

        X = torch.relu(self.fc1(X.reshape(len(inputs),-1)))
        X = torch.relu(self.fc2(X))
        X = self.output(X)


        return X

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5947234


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              29843
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 200_000_000
eval_every_steps = 10000
lr = 3.5e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=256,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29877.90, valid ppl: 28381.83
10000 steps; 10240000 examples so far; train ppl: 394.96, valid ppl: 235.44
20000 steps; 20480000 examples so far; train ppl: 201.82, valid ppl: 186.40
30000 steps; 30720000 examples so far; train ppl: 167.91, valid ppl: 172.69
40000 steps; 40960000 examples so far; train ppl: 152.56, valid ppl: 167.15
50000 steps; 51200000 examples so far; train ppl: 152.11, valid ppl: 161.15
60000 steps; 61440000 examples so far; train ppl: 140.05, valid ppl: 162.16
70000 steps; 71680000 examples so far; train ppl: 139.06, valid ppl: 158.98
80000 steps; 81920000 examples so far; train ppl: 142.09, valid ppl: 156.43
90000 steps; 92160000 examples so far; train ppl: 129.48, valid ppl: 157.99
100000 steps; 102400000 examples so far; train ppl: 134.04, valid ppl: 155.16
110000 steps; 112640000 examples so far; train ppl: 133.63, valid ppl: 156.68
120000 steps; 122880000 examples so far; train ppl: 125.95, valid ppl: 155.04
130000 steps;

In [ ]:
max_examples = 200_000_000
eval_every_steps = 10000
lr = 3.5e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30381.77, valid ppl: 29624.64
10000 steps; 10240000 examples so far; train ppl: 494.11, valid ppl: 285.91
20000 steps; 20480000 examples so far; train ppl: 240.65, valid ppl: 213.34
30000 steps; 30720000 examples so far; train ppl: 194.39, valid ppl: 190.34
40000 steps; 40960000 examples so far; train ppl: 173.43, valid ppl: 177.79
50000 steps; 51200000 examples so far; train ppl: 167.16, valid ppl: 170.31
60000 steps; 61440000 examples so far; train ppl: 155.77, valid ppl: 167.63
70000 steps; 71680000 examples so far; train ppl: 151.21, valid ppl: 162.89
80000 steps; 81920000 examples so far; train ppl: 151.03, valid ppl: 158.88
90000 steps; 92160000 examples so far; train ppl: 141.60, valid ppl: 158.34
100000 steps; 102400000 examples so far; train ppl: 142.80, valid ppl: 156.66
110000 steps; 112640000 examples so far; train ppl: 141.79, valid ppl: 155.51
120000 steps; 122880000 examples so far; train ppl: 135.91, valid ppl: 154.57
130000 steps;

In [ ]:
max_examples = 200_000_000
eval_every_steps = 10000
lr = 3.5e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=128, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=128)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30220.41, valid ppl: 29868.55
10000 steps; 1280000 examples so far; train ppl: 1145.20, valid ppl: 840.33
20000 steps; 2560000 examples so far; train ppl: 727.20, valid ppl: 666.98
30000 steps; 3840000 examples so far; train ppl: 605.01, valid ppl: 577.72
40000 steps; 5120000 examples so far; train ppl: 535.72, valid ppl: 517.52
50000 steps; 6400000 examples so far; train ppl: 489.16, valid ppl: 478.21
60000 steps; 7680000 examples so far; train ppl: 457.10, valid ppl: 451.80
70000 steps; 8960000 examples so far; train ppl: 436.16, valid ppl: 428.24
80000 steps; 10240000 examples so far; train ppl: 417.27, valid ppl: 415.16
90000 steps; 11520000 examples so far; train ppl: 401.49, valid ppl: 399.90
100000 steps; 12800000 examples so far; train ppl: 390.42, valid ppl: 386.86
110000 steps; 14080000 examples so far; train ppl: 380.88, valid ppl: 377.34
120000 steps; 15360000 examples so far; train ppl: 373.89, valid ppl: 369.10
130000 steps; 16640000

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 138.09844746539864


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 135.03568507247374


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 300.1787930254691


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de jogar jogos e assistir filmes de'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de jogar jogos e assistir filmes de um
Eu gosto de jogar jogos e assistir filmes de um dia
Eu gosto de jogar jogos e assistir filmes de um dia,
Eu gosto de jogar jogos e assistir filmes de um dia, mas
Eu gosto de jogar jogos e assistir filmes de um dia, mas o
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que é
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que é um
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que é um dos
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que é um dos que
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que é um dos que não
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que é um dos que não se
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que é um dos que não se pode
Eu gosto de jogar jogos e assistir filmes de um dia, mas o que é um dos que não se pode ser
Eu gosto de jogar jogos e assistir filmes de

In [ ]:
prompt = 'Eu gosto de jogar jogos e assistir filmes de'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de jogar jogos e assistir filmes de futebol
Eu gosto de jogar jogos e assistir filmes de futebol,
Eu gosto de jogar jogos e assistir filmes de futebol, o
Eu gosto de jogar jogos e assistir filmes de futebol, o que
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o primeiro
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o primeiro,
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o primeiro, o
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o primeiro, o que
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o primeiro, o que é
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o primeiro, o que é o
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o primeiro, o que é o que
Eu gosto de jogar jogos e assistir filmes de futebol, o que foi o primeiro, 

# Teste 4

In [ ]:
max_examples = 600_000_000
eval_every_steps = 10000
lr = 4.2e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=2048, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=2048)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29708.59, valid ppl: 28862.38
10000 steps; 20480000 examples so far; train ppl: 365.53, valid ppl: 218.11
20000 steps; 40960000 examples so far; train ppl: 185.59, valid ppl: 176.15
30000 steps; 61440000 examples so far; train ppl: 157.52, valid ppl: 159.55
40000 steps; 81920000 examples so far; train ppl: 144.38, valid ppl: 150.66
50000 steps; 102400000 examples so far; train ppl: 133.37, valid ppl: 146.03
60000 steps; 122880000 examples so far; train ppl: 128.00, valid ppl: 143.34
70000 steps; 143360000 examples so far; train ppl: 124.26, valid ppl: 140.32
80000 steps; 163840000 examples so far; train ppl: 120.83, valid ppl: 136.27
90000 steps; 184320000 examples so far; train ppl: 116.63, valid ppl: 135.12
100000 steps; 204800000 examples so far; train ppl: 114.61, valid ppl: 134.98
110000 steps; 225280000 examples so far; train ppl: 113.44, valid ppl: 133.18
120000 steps; 245760000 examples so far; train ppl: 111.46, valid ppl: 131.60
130000 s

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 114.54281655181148


In [ ]:
prompt = 'Eu gosto de jogar jogos e assistir filmes de'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de jogar jogos e assistir filmes de filmes
Eu gosto de jogar jogos e assistir filmes de filmes,
Eu gosto de jogar jogos e assistir filmes de filmes, mas
Eu gosto de jogar jogos e assistir filmes de filmes, mas o
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que é
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que é um
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que é um dos
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que é um dos mais
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que é um dos mais importantes
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que é um dos mais importantes do
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que é um dos mais importantes do mundo
Eu gosto de jogar jogos e assistir filmes de filmes, mas o que é um dos mais importantes do mundo.
Eu gosto de jogar jogos e assistir filmes de filmes, mas

In [ ]:
max_examples = 100_000_000
eval_every_steps = 10000
lr = 4.2e-4

train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 106.90, valid ppl: 127.23
10000 steps; 20480000 examples so far; train ppl: 97.59, valid ppl: 126.17
20000 steps; 40960000 examples so far; train ppl: 97.48, valid ppl: 127.41
30000 steps; 61440000 examples so far; train ppl: 97.56, valid ppl: 126.70
40000 steps; 81920000 examples so far; train ppl: 97.74, valid ppl: 125.88


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 113.8319469185642


In [ ]:
prompt = 'Eu gosto de jogar jogos e assistir filmes de'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de jogar jogos e assistir filmes de terror
Eu gosto de jogar jogos e assistir filmes de terror.
Eu gosto de jogar jogos e assistir filmes de terror. O
Eu gosto de jogar jogos e assistir filmes de terror. O que
Eu gosto de jogar jogos e assistir filmes de terror. O que é
Eu gosto de jogar jogos e assistir filmes de terror. O que é um
Eu gosto de jogar jogos e assistir filmes de terror. O que é um dos
Eu gosto de jogar jogos e assistir filmes de terror. O que é um dos mais
Eu gosto de jogar jogos e assistir filmes de terror. O que é um dos mais importantes
Eu gosto de jogar jogos e assistir filmes de terror. O que é um dos mais importantes do
Eu gosto de jogar jogos e assistir filmes de terror. O que é um dos mais importantes do mundo
Eu gosto de jogar jogos e assistir filmes de terror. O que é um dos mais importantes do mundo,
Eu gosto de jogar jogos e assistir filmes de terror. O que é um dos mais importantes do mundo, e
Eu gosto de jogar jogos e assistir filmes de terror. O q

In [ ]:
max_examples = 100_000_000
eval_every_steps = 10000
lr = 4.2e-4

train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 108.49, valid ppl: 126.35
10000 steps; 20480000 examples so far; train ppl: 96.16, valid ppl: 125.98
20000 steps; 40960000 examples so far; train ppl: 96.21, valid ppl: 126.70
30000 steps; 61440000 examples so far; train ppl: 96.26, valid ppl: 127.66
40000 steps; 81920000 examples so far; train ppl: 96.61, valid ppl: 125.71


In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 113.99603483379168


In [ ]:
prompt = 'Eu gosto de jogar jogos e assistir filmes de'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de jogar jogos e assistir filmes de "
Eu gosto de jogar jogos e assistir filmes de " O
Eu gosto de jogar jogos e assistir filmes de " O '
Eu gosto de jogar jogos e assistir filmes de " O'Neil
Eu gosto de jogar jogos e assistir filmes de " O'Neill
Eu gosto de jogar jogos e assistir filmes de " O'Neill "
Eu gosto de jogar jogos e assistir filmes de " O'Neill ",
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que é
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que é o
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que é o que
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que é o que é
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que é o que é o
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que é o que é o que
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que é o que é o que é
Eu gosto de jogar jogos e assistir filmes de " O'Neill ", que é